In [1]:
! pip install -r requirements.txt

In [2]:
! pip install cohere langchain qdrant-client tfds-nightly python-dotenv pymongo pandas numpy modal modal-client

In [6]:

from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate, LLMChain
import textwrap as tr
import random
import dotenv
import os
import pymongo
import pandas as pd
from tqdm import tqdm

dotenv.load_dotenv(".env")

COHERE_API_KEY = os.getenv("COHERE_API_KEY")
MONGO_URI = os.getenv("MONGO_URI")

mongodb_client = pymongo.MongoClient(MONGO_URI, serverSelectionTimeoutMS=3_600_000)

# Create dataset and collection
db = mongodb_client.cupidgpt
collection = db.profiles

### Load data into MongoDB

In [4]:
df = pd.read_csv("data/profile_cleaned.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/profile_cleaned.csv'

In [ ]:
# Save dataframe in MongoDB
mongodb_client = pymongo.MongoClient(MONGO_URI, serverSelectionTimeoutMS=3_600_000)

# Create dataset and collection
db = mongodb_client.cupidgpt
collection = db.profiles

# Insert dataframe into collection
# Insert in batches of 10
# for i in range(0, len(df), 10):
#     if i+10 < len(df):
#         df_batch = df[i:i+10]
#     else:
#         df_batch = df[i:]
#     collection.insert_many(df_batch.to_dict("records"))
collection.insert_many(df.to_dict("records"))

# Delete all documents in collection
# collection.delete_many({})

# Get number of items in collection
collection.count_documents({})



In [ ]:
# Get user_id 35 from collection
print(collection.find_one({"User_ID": 35}))


{'_id': ObjectId('64ea6a799c9fa34cea272636'), 'age': 27, 'status': 'single', 'sex': 'male', 'orientation': 'straight', 'body_type': 'thin', 'diet': 'strictly anything', 'drinks': 'drinks socially', 'drugs': 'never does drugs', 'education': 'graduated from college/university', 'ethnicity': 'white', 'height': 69.0, 'income': -1, 'job': 'entertainment / media', 'last_online': '2012-06-24-22-10', 'location': 'san francisco, california', 'offspring': 'Not Available', 'pets': 'likes dogs and likes cats', 'religion': 'atheism and somewhat serious about it', 'sign': 'sagittarius but it doesnot matter', 'smokes': 'smokes when drinking', 'speaks': 'english', 'About_Me': "i'm a cinematic artist for a video game studio. i love being in a creative environment surrounded by so many talented coworkers who challenge me to always be improving my work. this job has cultivated a strong passion for storytelling which has given me a bigger appetite for movies, books, and tv. i love discussing these and wha

### Cohere GenAI LLM - Test

In [13]:
user_profile = """
Create a descriptive Bio or summary for this user of a dating app. 
Write it like the person would have written about themselves. Talk about the persons features too
Do not include the kind of person they are looking for. Give me 200 words of description
Use all fields from the following JSON data.
User Data: {user_data}
"""

prompt = PromptTemplate(template=user_profile, input_variables=["user_data"])

llm = Cohere(cohere_api_key=COHERE_API_KEY, model="command", max_tokens=1000, temperature=0.8)
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Get user_id 35 from collection
user_data = collection.find_one({"User_ID": 35})
print(llm_chain.run(user_data=user_data))




 I am a 27-year-old male who is single and straight. I have a thin body type and I follow a strict diet. I drink socially and do not smoke or do drugs. I have a college degree and I work in the entertainment industry. I am white and have a Sagittarius sign. I am 5'9" and have a good income. I love to discuss movies, books, and TV shows. I am a passionate storyteller and I love being in a creative environment. I am always improving my work and I love being challenged by my coworkers. I am a cinematic artist for a video game studio and I love what I do. I am always up for a good discussion and I love to meet new people. I am a friendly and outgoing person and I am always up for a good time. I am a great listener and I am always there for my friends and family. I am a cinematic artist for a video game studio and I love what I do. I am always up for a good discussion and I love to meet new people. I am a friendly and outgoing person and I am always up for a good time. I love to make people

In [22]:
match_profile = """
Imagine you're the following user of a dating app with the following fields. Do not talk about yourself or the user. Talk about the relationship or the person
they are looking for. I want you to be creative and generate a 200 word response of the 
quality and attributes this person would like to marry or date. Specify what they are looking for and do not talk about the user itself.
User Data: {user_data}
"""

prompt_profile = PromptTemplate(template=match_profile, input_variables=["user_data"])

llm = Cohere(cohere_api_key=COHERE_API_KEY, model="command", max_tokens=1000, temperature=0.8)
llm_chain_profile = LLMChain(prompt=prompt_profile, llm=llm)

# Get user_id 35 from collection
finder_data = collection.find_one({"User_ID": 35})
print(llm_chain_profile.run(user_data=user_data))

 "Looking for a partner who is creative, passionate, and loves a good challenge. This person should be able to express themselves clearly and have a strong passion for storytelling. They should be able to discuss movies, books, and TV shows with ease and be able to explain why they love or hate a particular story. They should also be able to talk about the lighting and camera work in a scene and how it helped to sell the story. This person should be comfortable in a creative environment and be able to work well with others. They should also be able to handle criticism well and be able to use it to improve their work. They should be a good listener and be able to respect the opinions of others. They should be open-minded and be able to try new things. They should be a good communicator and be able to express their thoughts and feelings clearly. They should also be able to compromise and be willing to try things that they may not necessarily agree with. They should be a good friend and b

### Cohere GenAI LLM - Scale

In [ ]:

# see all User_ID in collections
about_me_generated = []
# use tqdm

for user in tqdm(collection.find()):
    # print(user["User_ID"])


    user_data = collection.find_one({"User_ID": user["User_ID"]})
    about_me_generated.append(llm_chain.run(user_data=user_data))
    # if user is last
    # if user["User_ID"] == collection.count_documents({}):
    #     print(llm_chain.output)

### Modal Encoding

In [12]:
import requests
from modal import Stub, Image, method

stub = Stub()

def download_models():
    # Caches the model inside the Modal image, so subsequent cold starts are faster.
    from sentence_transformers import SentenceTransformer
    SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

image = Image.debian_slim().pip_install("sentence-transformers").run_function(download_models)

@stub.cls(image=image)
class ModalEmbedding:
    def __enter__(self):
      from sentence_transformers import SentenceTransformer
      self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    @method()
    def generate(self, text: str):
      return self.model.encode(text).tolist()

In [11]:
import modal

stub = modal.Stub("example-get-started")


@stub.function()
def square(x):
    print("This code is running on a remote worker!")
    return x**2


@stub.local_entrypoint()
def main():
    print("the square is", square.remote(42))


In [12]:
stub.run("main")